# Online Prediction with scikit-learn on Google Cloud Machine Learning Engine
This notebook uses the [Census Income Data Set](https://archive.ics.uci.edu/ml/datasets/Census+Income) to create a simple model, train the model, upload the model to Cloud Machine Learning Engine (ML Engine), and lastly use the model to make predictions.

# How to bring your model to ML Engine
Getting your model ready for predictions can be done in 5 steps:
1. Save your model to a file
1. Upload the saved model to [Google Cloud Storage](https://cloud.google.com/storage)
1. Create a model resource on ML Engine
1. Create a model version (linking your scikit-learn model)
1. Make an online prediction

# Prerequisites
Before you jump in, let’s cover some of the different tools you’ll be using to get online prediction up and running on ML Engine. 

[Google Cloud Platform](https://cloud.google.com/) lets you build and host applications and websites, store data, and analyze data on Google's scalable infrastructure.

[Cloud ML Engine](https://cloud.google.com/ml-engine/) is a managed service that enables you to easily build machine learning models that work on any type of data, of any size.

[Google Cloud Storage](https://cloud.google.com/storage/) (GCS) is a unified object storage for developers and enterprises, from live data serving to data analytics/ML to data archiving.

[Cloud SDK](https://cloud.google.com/sdk/) is a command line tool which allows you to interact with Google Cloud products. In order to run this notebook, make sure that Cloud SDK is [installed](https://cloud.google.com/sdk/downloads) in the same environment as your Jupyter kernel.


# Part 0: Setup
* [Create a project on GCP](https://cloud.google.com/resource-manager/docs/creating-managing-projects)
* [Create a Google Cloud Storage Bucket](https://cloud.google.com/storage/docs/quickstart-console)
* [Enable Cloud Machine Learning Engine and Compute Engine APIs](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component&_ga=2.217405014.1312742076.1516128282-1417583630.1516128282)
* [Install Cloud SDK](https://cloud.google.com/sdk/downloads)
* [Install scikit-learn](http://scikit-learn.org/stable/install.html)
* [Install NumPy](https://docs.scipy.org/doc/numpy/user/install.html)
* [Install pandas](https://pandas.pydata.org/pandas-docs/stable/install.html)
* [Install Google API Python Client](https://github.com/google/google-api-python-client)

These variables will be needed for the following steps.

** Replace: **
* `PROJECT_ID <YOUR_PROJECT_ID>` - with your project's id. Use the PROJECT_ID that matches your Google Cloud Platform project.
* `BUCKET_ID <YOUR_BUCKET_ID>` - with the bucket id you created above.
* `MODEL_NAME <YOUR_MODEL_NAME>` - with your model name, such as '`census`'
* `VERSION <YOUR_VERSION>` - with your version name, such as '`v1`'
* `REGION <REGION>` - select a region from [here](https://cloud.google.com/ml-engine/docs/regions) or use the default '`us-central1`'. The region is where the model will be deployed.

In [1]:
%env PROJECT_ID true-ability-192918
%env BUCKET_ID true-ability-192918
%env MODEL_NAME census
%env VERSION_NAME v1
%env REGION us-central1

env: PROJECT_ID=true-ability-192918
env: BUCKET_ID=true-ability-192918
env: MODEL_NAME=census
env: VERSION_NAME=v1
env: REGION=us-central1


## Download the data
The [Census Income Data Set](https://archive.ics.uci.edu/ml/datasets/Census+Income) that this sample
uses for training is hosted by the [UC Irvine Machine Learning
Repository](https://archive.ics.uci.edu/ml/datasets/).

 * Training file is `adult.data`
 * Evaluation file is `adult.test`


### Disclaimer
This dataset is provided by a third party. Google provides no representation,
warranty, or other guarantees about the validity or any other aspects of this dataset.

In [2]:
# Create a directory to hold the data
! mkdir census_data

# Download the data
! curl https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data --output census_data/adult.data
! curl https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test --output census_data/adult.test

mkdir: census_data: File exists
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 3881k  100 3881k    0     0  4167k      0 --:--:-- --:--:-- --:--:-- 4168k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1956k  100 1956k    0     0  6466k      0 --:--:-- --:--:-- --:--:-- 6477k


# Part 1: Train/Save the model 
First, the data is loaded into a pandas DataFrame that can be used by scikit-learn. Then a simple model is created and fit against the training data. Lastly, sklearn's built in version of joblib is used to save the model to a file that can be uploaded to ML Engine.

In [3]:
import googleapiclient.discovery
import json
import numpy as np
import os
import pandas as pd
import pickle
from sklearn.ensemble import RandomForestClassifier
from sklearn.externals import joblib
from sklearn.feature_selection import SelectKBest
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelBinarizer

# Define the format of your input data including unused columns (These are the columns from the census data files)
COLUMNS = (
    'age',
    'workclass',
    'fnlwgt',
    'education',
    'education-num',
    'marital-status',
    'occupation',
    'relationship',
    'race',
    'sex',
    'capital-gain',
    'capital-loss',
    'hours-per-week',
    'native-country',
    'income-level'
)

# Categorical columns are columns that need to be turned into a numerical value to be used by scikit-learn
CATEGORICAL_COLUMNS = (
    'workclass',
    'education',
    'marital-status',
    'occupation',
    'relationship',
    'race',
    'sex',
    'native-country'
)


# Load the training census dataset
with open('./census_data/adult.data', 'r') as train_data:
    raw_training_data = pd.read_csv(train_data, header=None, names=COLUMNS)

# Remove the column we are trying to predict ('income-level') from our features list
# Convert the Dataframe to a lists of lists
train_features = raw_training_data.drop('income-level', axis=1).as_matrix().tolist()
# Create our training labels list, convert the Dataframe to a lists of lists
train_labels = (raw_training_data['income-level'] == ' >50K').as_matrix().tolist()


# Load the test census dataset
with open('./census_data/adult.test', 'r') as test_data:
    raw_testing_data = pd.read_csv(test_data, names=COLUMNS, skiprows=1)
# Remove the column we are trying to predict ('income-level') from our features list
# Convert the Dataframe to a lists of lists
test_features = raw_testing_data.drop('income-level', axis=1).as_matrix().tolist()
# Create our training labels list, convert the Dataframe to a lists of lists
test_labels = (raw_testing_data['income-level'] == ' >50K.').as_matrix().tolist()


# Since the census data set has categorical features, we need to convert
# them to numerical values. We'll use a list of pipelines to convert each
# categorical column and then use FeatureUnion to combine them before calling
# the RandomForestClassifier.
categorical_pipelines = []

# Each categorical column needs to be extracted individually and converted to a numerical value.
# To do this, each categorical column will use a pipeline that extracts one feature column via
# SelectKBest(k=1) and a LabelBinarizer() to convert the categorical value to a numerical one.
# A scores array (created below) will select and extract the feature column. The scores array is
# created by iterating over the COLUMNS and checking if it is a CATEGORICAL_COLUMN.
for i, col in enumerate(COLUMNS[:-1]):
    if col in CATEGORICAL_COLUMNS:
        # Create a scores array to get the individual categorical column.
        # Example:
        #  data = [39, 'State-gov', 77516, 'Bachelors', 13, 'Never-married', 'Adm-clerical', 
        #         'Not-in-family', 'White', 'Male', 2174, 0, 40, 'United-States']
        #  scores = [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        #
        # Returns: [['Sate-gov']]
        scores = []
        # Build the scores array
        for j in range(len(COLUMNS[:-1])):
            if i == j: # This column is the categorical column we want to extract.
                scores.append(1) # Set to 1 to select this column
            else: # Every other column should be ignored.
                scores.append(0)
        skb = SelectKBest(k=1)
        skb.scores_ = scores
        # Convert the categorical column to a numerical value
        lbn = LabelBinarizer()
        r = skb.transform(train_features)
        lbn.fit(r)
        # Create the pipeline to extract the categorical feature
        categorical_pipelines.append(
            ('categorical-{}'.format(i), Pipeline([
                ('SKB-{}'.format(i), skb),
                ('LBN-{}'.format(i), lbn)])))

# Create pipeline to extract the numerical features
skb = SelectKBest(k=6)
# From COLUMNS use the features that are numerical
skb.scores_ = [1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0]
categorical_pipelines.append(('numerical', skb))

# Combine all the features using FeatureUnion
preprocess = FeatureUnion(categorical_pipelines)

# Create the classifier
classifier = RandomForestClassifier()

# Transform the features and fit them to the classifier
classifier.fit(preprocess.transform(train_features), train_labels)

# Create the overall model as a single pipeline
pipeline = Pipeline([
    ('union', preprocess),
    ('classifier', classifier)
])

# Export the model to a file
joblib.dump(pipeline, 'model.joblib')

print('Model trained and saved')

Model trained and saved


# Part 2: Upload the model
Next, you'll need to upload the model to your project's storage bucket in GCS. To use your model with ML Engine, it needs to be uploaded to Google Cloud Storage (GCS). This step takes your local ‘model.joblib’ file and uploads it GCS via the Cloud SDK using gsutil.

Before continuing, make sure you're [properly authenticated](https://cloud.google.com/sdk/gcloud/reference/auth/) and have [access to the bucket](https://cloud.google.com/storage/docs/access-control/). This next command sets your project to the one specified above.

Note: If you get an error below, make sure the Cloud SDK is installed in the kernel's environment.

In [4]:
! gcloud config set project $PROJECT_ID

Updated property [core/project].


Note: The exact file name of of the exported model you upload to GCS is important! Your model must be named  “model.joblib”, “model.pkl”, or “model.bst” with respect to the library you used to export it. This restriction ensures that the model will be safely reconstructed later by using the same technique for import as was used during export.

In [5]:
! gsutil cp ./model.joblib gs://$BUCKET_ID/model.joblib

Copying file://./model.joblib [Content-Type=application/octet-stream]...
- [1 files][  7.8 MiB/  7.8 MiB]                                                
Operation completed over 1 objects/7.8 MiB.                                      


# Part 3: Create a model resource
Cloud ML Engine organizes your trained models using model and version resources. A Cloud ML Engine model is a container for the versions of your machine learning model. For more information on model resources and model versions look [here](https://cloud.google.com/ml-engine/docs/deploying-models#creating_a_model_version). 

At this step, you create a container that you can use to hold several different versions of your actual model.

In [6]:
! gcloud ml-engine models create $MODEL_NAME --regions us-central1

Created ml engine model [projects/true-ability-192918/models/census].


# Part 4: Create a model version

Now it’s time to get your model online and ready for predictions. The model version requires a few components as specified [here](https://cloud.google.com/ml-engine/reference/rest/v1/projects.models.versions#Version).

* __name__ - The name specified for the version when it was created. This will be the `VERSION_NAME` variable you declared at the beginning.
* __deployment Uri__ (curl) or __origin__ (gcloud) - The Google Cloud Storage location of the trained model used to create the version. This is the bucket that you uploaded the model to with your `BUCKET_ID`
* __runtime__ version - The Google Cloud ML runtime version to use for this deployment. This is set to 1.4
* __framework__ - The framework specifies if you are using: `TENSORFLOW`, `SCIKIT_LEARN`, `XGBOOST`. This is set to `SCIKIT_LEARN`
* __pythonVersion__ - This specifies whether you’re using Python 2.7 or Python 3.5. The default value is set to `“2.7”`, if you are using Python 3.5, set the value to `“3.5”`


Note: It can take several minutes for you model to be available.

Note: If you require a feature of scikit-learn that isn’t available in the publicly released version yet, you can specify “runtimeVersion”: “HEAD” instead, and that would get the latest version of scikit-learn available from the github repo. Otherwise the following versions will be used:
* scikit-learn: 0.19.0

In [7]:
! curl -X POST -H "Content-Type: application/json" \
   -d '{"name": "'$VERSION_NAME'", "deploymentUri": "gs://'$BUCKET_ID'/", "runtimeVersion": "1.4", "framework": "SCIKIT_LEARN", "pythonVersion": "3.5"}' \
   -H "Authorization: Bearer `gcloud auth print-access-token`" \
    https://ml.googleapis.com/v1/projects/$PROJECT_ID/models/$MODEL_NAME/versions

{
  "name": "projects/true-ability-192918/operations/create_census_v1-1522340201314",
  "metadata": {
    "@type": "type.googleapis.com/google.cloud.ml.v1.OperationMetadata",
    "createTime": "2018-03-29T16:16:42Z",
    "operationType": "CREATE_VERSION",
    "modelName": "projects/true-ability-192918/models/census",
    "version": {
      "name": "projects/true-ability-192918/models/census/versions/v1",
      "deploymentUri": "gs://true-ability-192918/",
      "createTime": "2018-03-29T16:16:41Z",
      "runtimeVersion": "1.4",
      "framework": "SCIKIT_LEARN",
      "pythonVersion": "3.5"
    }
  }
}


# Part 5: Make an online prediction
It’s time to make an online prediction with your newly deployed model. Before you begin, you'll need to take some of the test data and prepare it, so that the test data can be used by the deployed model.

In [8]:
# Get one person that makes <=50K and one that makes >50K to test our model.
print('Show a person that makes <=50K:')
print('\tFeatures: {0} --> Label: {1}\n'.format(test_features[0], test_labels[0]))

with open('less_than_50K.json', 'w') as outfile:
  json.dump(test_features[0], outfile)

  
print('Show a person that makes >50K:')
print('\tFeatures: {0} --> Label: {1}'.format(test_features[3], test_labels[3]))

with open('more_than_50K.json', 'w') as outfile:
  json.dump(test_features[3], outfile)

Show a person that makes <=50K:
	Features: [25, ' Private', 226802, ' 11th', 7, ' Never-married', ' Machine-op-inspct', ' Own-child', ' Black', ' Male', 0, 0, 40, ' United-States'] --> Label: False

Show a person that makes >50K:
	Features: [44, ' Private', 160323, ' Some-college', 10, ' Married-civ-spouse', ' Machine-op-inspct', ' Husband', ' Black', ' Male', 7688, 0, 40, ' United-States'] --> Label: True


## Use gcloud to make online predictions
Use the two people (as seen in the table) gathered in the previous step for the gcloud predictions.

| **Person** | age | workclass | fnlwgt | education | education-num | marital-status | occupation |
|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:
| **1** | 25| Private | 226802 | 11th | 7 | Never-married | Machine-op-inspect |
| **2** | 44| Private | 160323 | Some-college | 10 | Married-civ-spouse | Machine-op-inspct |

| **Person** | relationship | race | sex | capital-gain | capital-loss | hours-per-week | native-country || (Label) income-level|
|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:||:-:
| **1** | Own-child | Black | Male | 0 | 0 | 40 | United-States || False (<=50K) |
| **2** | Huasband | Black | Male | 7688 | 0 | 40 | United-States || True (>50K) |


Creating a model version can take several minutes, check the status of your model version to see if it is available.

In [9]:
! gcloud ml-engine versions list --model $MODEL_NAME

NAME  DEPLOYMENT_URI             STATE
v1    gs://true-ability-192918/  CREATING


In [10]:
! gcloud ml-engine versions list --model $MODEL_NAME

NAME  DEPLOYMENT_URI             STATE
v1    gs://true-ability-192918/  READY


Test the model with an online prediction using the data of a person who makes <=50K.

Note: If you see an error, the model from Part 4 may not be created yet as it takes several minutes for a new model version to be created.

In [11]:
! gcloud ml-engine predict --model $MODEL_NAME --version $VERSION_NAME --json-instances less_than_50K.json

[False]


Test the model with an online prediction using the data of a person who makes >50K.

In [12]:
! gcloud ml-engine predict --model $MODEL_NAME --version $VERSION_NAME --json-instances more_than_50K.json

[True]


## Use Python to make online predictions
Test the model with the entire test set and print out some of the results.

Note: If running notebook server on Compute Engine, make sure to ["allow full access to all Cloud APIs".](https://cloud.google.com/compute/docs/access/create-enable-service-accounts-for-instances#changeserviceaccountandscopes)

In [13]:
PROJECT_ID = os.environ['PROJECT_ID']
VERSION_NAME = os.environ['VERSION_NAME']
MODEL_NAME = os.environ['MODEL_NAME']

service = googleapiclient.discovery.build('ml', 'v1')
name = 'projects/{}/models/{}'.format(PROJECT_ID, MODEL_NAME)
name += '/versions/{}'.format(VERSION_NAME)

# Due to the size of the data, it needs to be split in 2
first_half = test_features[:int(len(test_features)/2)]
second_half = test_features[int(len(test_features)/2):]

complete_results = []
for data in [first_half, second_half]:
    responses = service.projects().predict(
        name=name,
        body={'instances': data}
    ).execute()

    if 'error' in responses:
        print(response['error'])
    else:
        complete_results.extend(responses['predictions'])
        
# Print the first 10 responses
for i, response in enumerate(complete_results[:10]):
    print('Prediction: {}\tLabel: {}'.format(response, test_labels[i]))

Prediction: False	Label: False
Prediction: False	Label: False
Prediction: True	Label: True
Prediction: True	Label: True
Prediction: False	Label: False
Prediction: False	Label: False
Prediction: False	Label: False
Prediction: True	Label: True
Prediction: False	Label: False
Prediction: False	Label: False


# [Optional] Part 6: Verify Results
Use a confusion matrix to create a visualization of the online predicted results from ML Engine.

In [14]:
actual = pd.Series(test_labels, name='actual')
online = pd.Series(complete_results, name='online')

pd.crosstab(actual,online)

online,False,True
actual,,
False,11546,889
True,1673,2173


Use a confusion matrix create a visualization of the predicted results from the local model. These results should be identical to the results above.

In [15]:
local_results = pipeline.predict(test_features)
local = pd.Series(local_results, name='local')

pd.crosstab(actual,local)

local,False,True
actual,,
False,11546,889
True,1673,2173


Directly compare the two results

In [16]:
identical = 0
different = 0

for i in range(len(complete_results)):
    if complete_results[i] == local_results[i]:
        identical += 1
    else:
        different += 1
        
print('identical: {}, different: {}'.format(identical,different))

identical: 16281, different: 0


If all results are identical, it means you've successfully uploaded your local model to ML Engine and performed online predictions correctly.